In [1]:
import pandas as pd

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

Dataset URL: https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria
License(s): unknown
 98% 665M/675M [00:06<00:00, 173MB/s]
100% 675M/675M [00:06<00:00, 106MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cell_image.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [44]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [45]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

In [46]:
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable=True
  if set_trainable:
    layer.trainable = False
  else:
    layer.trainable = False


In [47]:
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

In [48]:
data = keras.utils.image_dataset_from_directory(
    directory = '/content/cell_images',
    labels='inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size=(150, 150)
)

Found 27558 files belonging to 2 classes.


In [49]:
# Calculate the number of batches for train and test
total_batches = tensorflow.data.experimental.cardinality(data).numpy()
train_size = int(0.8 * total_batches)

In [50]:
train_ds = data.take(train_size)
test_ds = data.skip(train_size)

In [51]:
def process(image, label):
  image = tensorflow.cast(image/255., tensorflow.float32)
  return (image, label)

train_ds = train_ds.map(process)
test_ds =  test_ds.map(process)

In [52]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
history = model.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 80s 113ms/step - accuracy: 0.8361 - loss: 0.4180 - val_accuracy: 0.9334 - val_loss: 0.1732
Epoch 2/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 78s 113ms/step - accuracy: 0.9288 - loss: 0.1843 - val_accuracy: 0.9187 - val_loss: 0.2122
Epoch 3/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 81s 112ms/step - accuracy: 0.9391 - loss: 0.1614 - val_accuracy: 0.9372 - val_loss: 0.1702
Epoch 4/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 82s 112ms/step - accuracy: 0.9441 - loss: 0.1488 - val_accuracy: 0.9439 - val_loss: 0.1491
Epoch 5/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 82s 112ms/step - accuracy: 0.9491 - loss: 0.1351 - val_accuracy: 0.9319 - val_loss: 0.1927


In [31]:
d